In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
gene_df[gene_df['name']=='NM_001385633']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
47734,Kcnc3,NM_001385633,chr7,+,44239875,44254175,44240309,44250309,5,"44239875,44244584,44247804,44250292,44251292,","44241182,44245692,44247999,44250352,44254175,"


In [9]:
def target_base_CT_plus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
def target_base_AG_plus(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][-3:]
        if target =='TGG':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [11]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [12]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [13]:
def exon_data(name,sequence_data):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)
        
    
    
    return ''.join(exon_seq_list)

In [14]:
def where_start_codon(name,exon_data):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsStart'].to_list()[0])
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
             exon_num =s
            
    return exon_num


In [15]:
def start_codon(name,exon_data,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    CDs =''
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    cds_list=[]
    if exon_num == 0:
            cds_list.append(exon_data[set_num:])
            
    
    
    else:     
        start_num = int(data['cdsStart'].to_list()[0])- int(start[exon_num])

        for s in range(exon_num):
            start_num += int(end[s])-int(start[s])
        cds_list.append(exon_data[start_num:])
        
    return ''.join(cds_list)

In [16]:
def end_codon(cds_seq):
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [17]:
def add_num_list(name):#修正が必要
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [18]:
def add_num(name,number_list, interval_list):
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
            
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    for i in indices:
        if i == 0:
            add_result.append(set_num)
        else:
            add_result.append(int(start[i]))
        
    return add_result

In [64]:
def CT_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_CT_plus(seq),seq))#配列的に狙えるほう
    
    
    
    start_exon_num= where_start_codon(name,exon_data(name,seq))
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name))
    
    print(len(start_codon(name,exon_data(name,seq),start_exon_num)))
    print(end_codon_num)
    print(add_list)
    
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)
    
    num_able = available_base_num
    num_codon = end_codon_num+add_list
    
    result = np.intersect1d(num_able,num_codon)
    print(num_able)
    print('a')
    print(num_codon)
    return result

In [65]:
CT_target_base_strand_plus("NM_001301868")

8127
[18, 30, 129, 132, 150, 183, 246, 294, 330, 435, 444, 465, 633, 636, 663, 816, 897, 954, 1005, 1056, 1113, 1140, 1176, 1251, 1293, 1329, 1455, 1521, 1542, 1572, 1584, 1611, 1635, 1734, 1908, 1917, 1950, 1962, 1971, 2004, 2013, 2022, 2076, 2079, 2091, 2190, 2250, 2283, 2295, 2355, 2394, 2433, 2454, 2478, 2589, 2592, 2664, 2694, 2715, 2745, 2775, 2799, 2820, 2829, 2859, 2907, 2937, 2940, 3027, 3210, 3246, 3264, 3291, 3297, 3360, 3459, 3501, 3645, 3672, 3720, 3729, 3735, 3756, 3759, 3852, 3864, 3885, 3978, 4125, 4134, 4194, 4197, 4200, 4245, 4305, 4404, 4443, 4446, 4629, 4638, 4647, 4665, 4746, 4779, 4827, 4839, 4920, 4938, 4950, 4971, 5001, 5061, 5073, 5127, 5139, 5265, 5409, 5454, 5475, 5541, 5661, 5664, 5691, 5721, 5739, 5760, 5793, 5799, 5814, 5883, 5928, 5961, 5982, 5991, 6003, 6033, 6111, 6123, 6222, 6279, 6291, 6417, 6453, 6465, 6486, 6552, 6570, 6582, 6603, 6633, 6693, 6705, 6759, 6771, 6897, 7041, 7086, 7107, 7173, 7293, 7296, 7323, 7353, 7371, 7392, 7425, 7431, 7446, 7515, 

ValueError: operands could not be broadcast together with shapes (184,) (117,) 

In [55]:
AG_target_base_strand_plus("NM_001385633")

1717
0
[44239972 44240103 44240478 44240807 44241366 44242511 44242526 44242645
 44243239 44243625 44244357 44244569 44244713 44245212 44245508 44245520
 44245637 44245680 44246557 44248188 44248242 44249744 44250212 44250348
 44250468 44250941 44251148 44252556 44252580 44252993 44253208 44253565]
a
[44240330 44240348 44240447 44240834 44240870 44240885 44241053 44241113
 44241116 44241125 44241128 44241140 44241182 44241191 44241203 44241287
 44241488 44241602 44241605 44245979 44246003 44246018 44246060 44246078
 44246084 44246093 44246117 44246195]


array([], dtype=int32)

In [62]:
AG_target_base_strand_plus("NM_001301868")

8127
1


ValueError: operands could not be broadcast together with shapes (184,) (117,) 

In [27]:
gene_df[gene_df['strand']=='+']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
12,Lypla1,NM_001355712,chr1,+,4878045,4916958,4878136,4915239,8,"4878045,4878677,4898806,4900490,4902533,490722...","4878205,4878709,4898872,4900538,4902604,490729..."
13,Lypla1,NM_008866,chr1,+,4878045,4916958,4878136,4915239,9,"4878045,4878677,4898806,4900490,4902533,490722...","4878205,4878709,4898872,4900538,4902604,490729..."
14,Tcea1,NM_001159750,chr1,+,4927916,4968132,4928136,4966587,10,"4927916,4937692,4948249,4956966,4959682,496096...","4928199,4937755,4948355,4957054,4959825,496101..."
15,Tcea1,NM_011541,chr1,+,4927916,4968132,4928136,4966587,10,"4927916,4937692,4948249,4956966,4959679,496096...","4928199,4937755,4948355,4957054,4959825,496101..."
16,Tcea1,NM_001159751,chr1,+,4928550,4968132,4928630,4966587,10,"4928550,4937692,4948249,4956966,4959679,496096...","4928726,4937755,4948355,4957054,4959825,496101..."
...,...,...,...,...,...,...,...,...,...,...,...
47734,Kcnc3,NM_001385633,chr7,+,44239875,44254175,44240309,44250309,5,"44239875,44244584,44247804,44250292,44251292,","44241182,44245692,44247999,44250352,44254175,"
47735,Kcnc3,NM_008422,chr7,+,44239875,44254175,44240309,44251429,4,"44239875,44244584,44247807,44251292,","44241182,44245692,44247999,44254175,"
47736,Kcnc3,NR_169735,chr7,+,44239875,44254175,44254175,44254175,6,"44239875,44244584,44247804,44248229,44250097,4...","44241182,44245692,44247999,44248357,44250352,4..."
47737,Kcnc3,NR_169736,chr7,+,44239875,44254175,44254175,44254175,6,"44239875,44244584,44247804,44248229,44250292,4...","44241182,44245692,44247999,44248357,44250352,4..."


In [61]:
gene_df.loc[5050]

geneName                                                  Ikzf1
name                                               NM_001301868
chrom                                                     chr11
strand                                                        +
txStart                                                11635924
txEnd                                                  11722926
cdsStart                                               11650218
cdsEnd                                                 11719581
exonCount                                                     8
exonStarts    11635924,11650204,11657784,11698310,11704006,1...
exonEnds      11636468,11650258,11657904,11698571,11704174,1...
Name: 5050, dtype: object

In [60]:
gene_df[gene_df['name']=='NM_008907']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
5000,Ppia,NM_008907,chr11,+,6365866,6369817,6365910,6369610,5,"6365866,6368100,6368225,6369112,6369477,","6365979,6368131,6368314,6369285,6369817,"
44294,Ppia,NM_008907,chr9,+,96777667,96778407,96777710,96778205,1,"96777667,","96778407,"


In [ ]:
def AG_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_AG_plus(seq),seq))#配列的に狙えるほう
    
    
    start_exon_num= where_start_codon(name,exon_data(name,seq))
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name))
    
    #print(start_codon(name,exon_data(name,seq),start_exon_num))
    print(len(start_codon(name,exon_data(name,seq),start_exon_num)))
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)
    
    print(start_exon_num)
    
    num_able = available_base_num
    num_codon = end_codon_num+add_list#GGXXXなので22くらい足さなきゃいけん要修正
    
    print(num_able)
    print('a')
    print(num_codon)
    result = np.intersect1d(num_able,num_codon)
    
    return result